In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np
import numpy.random as nprand
import pandas as pd
import random
import scipy.stats as spstats
%matplotlib inline

from loomio import *
from socialchoice import *
from netdelib import *

In [ ]:
class NetDelibInfluence(object):
    
    def __init__(self, df_score):
        
        # Separate control from random-pod
        self.df_control = df_score[df_score.treatment == 1]
        self.df_random = df_score[df_score.treatment == 2]
    
        # Create preference sequences
        self.control_preferences = make_preference_sequence_collection(self.df_control)
        self.random_preferences = make_preference_sequence_collection(self.df_random)
        
        # Calculate participant conformity
        self.control_conformity = list(self.conformity_all(self.control_preferences).values())
        self.random_conformity = list(self.conformity_all(self.random_preferences).values())

        # Calculate participant hipness
        self.control_hipness = list(self.hipness_all(self.control_preferences).values())
        self.random_hipness = list(self.hipness_all(self.random_preferences).values())
        

    def pair_conformity(self, p, q, start=0, end=3):
        '''Quantifies p's conformity toward q.'''
        initial = p[start].crossing_dissimilarity(q[start])
        final = p[end].crossing_dissimilarity(q[end])
        dp = p[start].crossing_dissimilarity(p[end])
        dq = q[start].crossing_dissimilarity(q[end])
        try:
            result = -1 * (final - initial) * (dp / (dq + dp))
        except ZeroDivisionError:
            # Both dq and dp are zero, no contribution to influence of p
            result = 0
        return result
        
        
    def conformity(self, participant_id, preferences, start=0, end=3):
        p = preferences[participant_id]
        conformity_q = []
        for q_id, q in preferences.items():
            if q_id == participant_id:
                # Avoid self-comparison
                continue
            conformity_q.append(self.pair_conformity(p, q, start, end))
        return np.mean(conformity_q)

    def conformity_all(self, preferences):
        results = {}
        for p_id, p in preferences.items():
            results[p_id] = self.conformity(p_id, preferences)
        return results
    
    def hipness(self, participant_id, preferences, start=0, end=3):
        p = preferences[participant_id]
        hipness_q = []
        for q_id, q in preferences.items():
            if q_id == participant_id:
                # Avoid self-comparison
                continue
            hipness_q.append(self.pair_conformity(q, p, start, end))
        return np.mean(hipness_q)
    
    def hipness_all(self, preferences):
        results = {}
        for p_id, p in preferences.items():
            results[p_id] = self.hipness(p_id, preferences)
        return results

    def plot_hipness_hist(self):
        plt.hist([self.control_hipness, self.random_hipness], label=["Control", "Random-Pod"])
        plt.legend()        
    
    def plot_conformity(self):
        x = np.array(sorted(self.control_conformity,reverse=True))
        plt.plot(range(1, len(x)+1), x, label="Control")
        x = np.array(sorted(self.random_conformity, reverse=True))
        plt.plot(range(1, len(x)+1), x, label="Random-Pod")
        plt.xlabel('Participant')
        plt.ylabel('Conformity')
        plt.xticks([1, 9, 17, 26, 33])
        plt.ylim([-0.25, 0.25])
        plt.grid()
        plt.legend()
        
    def plot_conformity_cumulative(self):
        x = np.array(sorted(self.control_conformity,reverse=True))
        plt.plot([0] + [sum(x[:i+1]) for i, xi in enumerate(x)], label="Control")
        x = np.array(sorted(self.random_conformity, reverse=True))
        plt.plot([0] + [sum(x[:i+1]) for i, xi in enumerate(x)], label="Random-Pod")
        plt.xlabel('Participant')
        plt.ylabel('Conformity (cumulative)')
        plt.xticks([1, 9, 17, 26, 33])
        plt.ylim([-1.25, 1.25])
        plt.grid()
        plt.legend()
        
    def plot_hipness(self):
        x = np.array(sorted(self.control_hipness,reverse=True))
        plt.plot(range(1, len(x)+1), x, label="Control")
        x = np.array(sorted(self.random_hipness, reverse=True))
        plt.plot(range(1, len(x)+1), x, label="Random-Pod")
        plt.xticks([1, 9, 17, 26, 33])
        plt.ylim([-0.15, 0.15])
        plt.xlabel('Participant')
        plt.ylabel('Hipness')
        plt.grid()
        plt.legend()
        
    def plot_hipness_cumulative(self):
        x = np.array(sorted(self.control_hipness,reverse=True))
        plt.plot([0] + [sum(x[:i+1]) for i, xi in enumerate(x)], label="Control")
        x = np.array(sorted(self.random_hipness, reverse=True))
        plt.plot([0] + [sum(x[:i+1]) for i, xi in enumerate(x)], label="Random-Pod")
        plt.xticks([1, 9, 17, 26, 33])
        plt.ylim([-0.75, 0.75])
        plt.xlabel('Participant')
        plt.ylabel('Hipness (cumulative)')
        plt.grid()
        plt.legend()



In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = fill_attrition(df_score)

netdelib = NetDelibInfluence(df_score)

In [ ]:
netdelib.plot_hipness_hist()

In [ ]:
plt.figure(figsize=(8,4))

plt.subplot(1,2,1)
netdelib.plot_conformity()

plt.subplot(1,2,2)
netdelib.plot_conformity_cumulative()

plt.tight_layout()
plt.savefig('fig-conformity.eps')

In [ ]:
plt.figure(figsize=(8,4))

plt.subplot(1,2,1)
netdelib.plot_hipness()

plt.subplot(1,2,2)
netdelib.plot_hipness_cumulative()

plt.tight_layout()
plt.savefig('fig-hipness.eps')

In [ ]:
sum(netdelib.control_conformity)

In [ ]:
sum(netdelib.random_conformity)